In [30]:

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np

In [31]:
lending_club_df = pd.read_csv("./data/final_lending_club.csv")

In [32]:
lending_club_df.head()

,Unnamed: 0,loan_amnt,funded_amnt,funded_amnt_inv,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,...,addr_state_SD,addr_state_TN,addr_state_TX,addr_state_UT,addr_state_VA,addr_state_VT,addr_state_WA,addr_state_WI,addr_state_WV,addr_state_WY
0,0,28000.0,28000.0,28000.0,7.62,872.52,5,325000.0,18.55,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1,27600.0,27600.0,27600.0,19.97,730.78,6,73000.0,23.13,1.0,...,0,0,0,0,0,0,0,0,0,0
2,3,24000.0,24000.0,24000.0,13.53,814.80,10,100000.0,22.18,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,8000.0,8000.0,8000.0,10.99,261.88,2,33000.0,15.75,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,11500.0,11500.0,11500.0,22.90,323.54,4,32760.0,27.06,0.0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
# target column
lending_club_df.repaid.head()

0    1
1    0
2    1
3    0
4    0
Name: repaid, dtype: int64

In [34]:
y_col = lending_club_df.repaid

In [35]:
x_cols = lending_club_df.ix[:,lending_club_df.columns!='repaid'] 

In [36]:
X_train, X_test, y_train, y_test = train_test_split(x_cols, y_col,test_size=.15)

In [37]:
X_t, X_val, y_t, y_val = train_test_split(X_train, y_train,test_size=.15)

In [38]:
X_t.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 961071 entries, 105977 to 388643
Columns: 160 entries, Unnamed: 0 to addr_state_WY
dtypes: float64(74), int64(86)
memory usage: 1.2 GB


# Random Forrest First

In [ ]:
rf = RandomForestClassifier(n_estimators=100)

In [ ]:
rf.fit(X_t,y_t)

In [ ]:
accuracy_score(y_val, rf.predict(X_val))

In [ ]:
lending_club_df.columns[np.argsort(rf.feature_importances_)[::-1]][:20]